In [1]:
import time
import os, zipfile, requests, pandas as pd, geopandas as gpd, osmnx as ox, networkx as nx
import ast
import statistics
import numpy as np
from sklearn.neighbors import BallTree
from shapely.geometry import Point
import random
import matplotlib.path as mpltPath
import json 

ox.settings.use_cache = True
ox.settings.log_console = True
print('ox {}\nnx {}'.format(ox.__version__, nx.__version__))
start_time = time.time()

ox 2.0.0
nx 3.4.2


# Get all TAZs for the Bay Area and all trips from the Bay Area MTC

In [2]:
taz_loc_df = pd.read_csv("/home/rishi/Berkeley/LPSim/LPSim/transportation_analysis_zones_1454_3081571714501117281.csv")
trips_df = pd.read_csv("/home/rishi/Berkeley/LPSim/LPSim/sf_mtc_od.csv")

In [3]:
trips_df.head()

,x_x,y_x,x_y,y_y,time_departure
0,-122.397449,37.792642,-122.402791,37.792859,36555.8
1,-122.398992,37.794292,-122.400959,37.792116,33181.4
2,-122.396480,37.793665,-122.400959,37.792116,31361.4
3,-122.398992,37.794292,-122.401152,37.793067,28975.4
4,-122.397404,37.792440,-122.399785,37.792261,27926.8


# Narrow down trips to only morning trips between 8-9am (morning peak) and also only to car trips (both driving and ridesharing)

In [ ]:
#get only peak morning trips and the shape
depart_hour = 8
morning_peak = trips_df[
    (trips_df.time_departure >= (depart_hour) * 3600) & 
    (trips_df.time_departure <= (depart_hour + 2) * 3600)
] #| (trips_df.depart_hour == 9)]
morning_peak.shape

time_string = '{}to{}'.format(depart_hour, depart_hour + 1)


In [6]:
#get full car trips from O to D (this includes ridesharing, but does not include car to another form of transit)
#morning_peak = morning_peak[morning_peak.trip_mode < 7]
morning_peak.shape

(56447, 5)

In [7]:
morning_peak.head()

,x_x,y_x,x_y,y_y,time_departure
3,-122.398992,37.794292,-122.401152,37.793067,28975.4
43,-122.398610,37.792413,-122.406982,37.789436,28891.3
57,-122.398298,37.791734,-122.406794,37.788501,28919.4
84,-122.396401,37.793249,-122.408275,37.778852,28954.0
159,-122.398298,37.791734,-122.389431,37.788852,28894.3


# Create functions to find the nodes that are within the polygons of each TAZ

In [8]:
def find_in_nodes(row, points, nodes_df):
    ### return the indices of points in nodes_df that are contained in row['geometry']
    if row['geometry'].type == 'MultiPolygon':
        return []
    else:
        path = mpltPath.Path(list(zip(*row['geometry'].exterior.coords.xy)))
        in_index = path.contains_points(points)
        return nodes_df['osmid'].loc[in_index].tolist()


def taz_nodes():
    ### Find corresponding nodes for each TAZ
    ### Input 1: TAZ polyline
    taz_gdf = gpd.read_file("/home/rishi/Berkeley/LPSim/LPSim/transportation_analysis_zones_1454.shp")
    #taz_gdf = gpd.read_file("san_francisco_taz.json")
    taz_gdf = taz_gdf.to_crs({'init': 'epsg:4326'})

    ### Input 2: OSM nodes coordinate
    nodes_df = pd.read_csv('../new_full_network/nodes.csv') ### `nodes.csv` from OSMNX
    points = nodes_df[['x', 'y']].values ### x, y are the coordinates of the nodes
    taz_gdf['in_nodes'] = taz_gdf.apply(lambda row: find_in_nodes(row, points, nodes_df), axis=1)
    taz_nodes_dict = {row['taz1454']:row['in_nodes'] for index, row in taz_gdf.iterrows()}
    
    return taz_nodes_dict
    ### [{'taz': 1, 'in_nodes': '[...]''}, ...]
    #with open('taz_nodes.json', 'w') as outfile:
    #    json.dump(taz_nodes_dict, outfile, indent=2)

In [9]:
taz_nodes_dict = taz_nodes()

/home/rishi/miniconda3/envs/lpsim/lib/python3.13/site-packages/pyogrio/raw.py:198: RuntimeWarning: /home/rishi/Berkeley/LPSim/LPSim/transportation_analysis_zones_1454.shp contains polygon(s) with rings with invalid winding order. Autocorrecting them, but that shapefile should be corrected using ogr2ogr for example.
  return ogr_read(
/home/rishi/miniconda3/envs/lpsim/lib/python3.13/site-packages/pyproj/crs/crs.py:143: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  in_crs_string = _prepare_from_proj_string(in_crs_string)
/tmp/ipykernel_423186/3531716854.py:3: ShapelyDeprecationWarning: The 'type' attribute is deprecated, and will be removed in the future. You can use the 'geom_type' attribute instead.
  if row['geometry'].type == 'MultiPolygon':


# Count the number of TAZs that do not have any nodes in its network (for informational purposes/sanity check)

In [10]:
#number of TAZ that do not have any nodes in its network
count = 0
for key, value in taz_nodes_dict.items():
    if len(taz_nodes_dict[key]) == 0:
        count += 1
print(count)

34


# Create functions to (a) generate a dataframe containing all OD from TAZ to TAZ, and then (b) randomly assign each O to a particular node in the TAZ and each D to a particular node in the TAZ

In [11]:
#create dataframe of each person with origin TAZ and destination TAZ
def create_od_df(trips_df):
    orig_dest_df = pd.DataFrame(columns=['orig', 'dest'])
    orig_dest_df['orig'] = morning_peak['orig_taz']
    orig_dest_df['dest'] = morning_peak['dest_taz']
    orig_dest_df['dep_time'] = morning_peak['time_departure']
    #orig_dest_df['person_num'] = morning_peak['person_num']
    return orig_dest_df

def assign_node_to_od(od_df, taz_nodes_dict, assign='random'):
    origs = []
    dests = []
    bad_list = []
    for i, row in enumerate(od_df.itertuples(), 0):
        len_network_o = len(taz_nodes_dict[row.orig])
        len_network_d = len(taz_nodes_dict[row.dest])
        if len_network_o == 0 or len_network_d == 0:
            bad_list += [i,]
        else:
            if assign == 'random':
                node_o = random.choice(taz_nodes_dict[row.orig])
                node_d = random.choice(taz_nodes_dict[row.dest])
                #print(node_o)
                #print(node_d)
        
                origs += [node_o,]
                dests += [node_d,]
                
    print("number of OD taz with zero nodes = {}".format(len(bad_list)))
    return origs, dests, bad_list

In [12]:
taz_gdf = gpd.read_file("/home/rishi/Berkeley/LPSim/LPSim/transportation_analysis_zones_1454.shp")
taz_gdf = taz_gdf.to_crs(epsg=4326)  # Ensure it matches your coordinates' CRS

/home/rishi/miniconda3/envs/lpsim/lib/python3.13/site-packages/pyogrio/raw.py:198: RuntimeWarning: /home/rishi/Berkeley/LPSim/LPSim/transportation_analysis_zones_1454.shp contains polygon(s) with rings with invalid winding order. Autocorrecting them, but that shapefile should be corrected using ogr2ogr for example.
  return ogr_read(


In [13]:
# Convert origin and destination coordinates to Points
morning_peak['orig_geometry'] = morning_peak.apply(lambda row: Point(row['x_x'], row['y_x']), axis=1)
morning_peak['dest_geometry'] = morning_peak.apply(lambda row: Point(row['x_y'], row['y_y']), axis=1)

/tmp/ipykernel_423186/1794825078.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  morning_peak['orig_geometry'] = morning_peak.apply(lambda row: Point(row['x_x'], row['y_x']), axis=1)
/tmp/ipykernel_423186/1794825078.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  morning_peak['dest_geometry'] = morning_peak.apply(lambda row: Point(row['x_y'], row['y_y']), axis=1)


In [14]:
orig_gdf = gpd.GeoDataFrame(morning_peak, geometry='orig_geometry', crs="EPSG:4326")
dest_gdf = gpd.GeoDataFrame(morning_peak, geometry='dest_geometry', crs="EPSG:4326")

# Spatial join to find TAZs
orig_with_taz = gpd.sjoin(orig_gdf, taz_gdf[['taz1454', 'geometry']], how="left", predicate='intersects')
dest_with_taz = gpd.sjoin(dest_gdf, taz_gdf[['taz1454', 'geometry']], how="left", predicate='intersects')

# Add TAZ columns to morning_peak DataFrame
morning_peak['orig_taz'] = orig_with_taz['taz1454']
morning_peak['dest_taz'] = dest_with_taz['taz1454']

/tmp/ipykernel_423186/2371353421.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  morning_peak['orig_taz'] = orig_with_taz['taz1454']
/tmp/ipykernel_423186/2371353421.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  morning_peak['dest_taz'] = dest_with_taz['taz1454']


In [15]:
print(morning_peak)

                x_x        y_x         x_y        y_y  time_departure  \
3       -122.398992  37.794292 -122.401152  37.793067         28975.4   
43      -122.398610  37.792413 -122.406982  37.789436         28891.3   
57      -122.398298  37.791734 -122.406794  37.788501         28919.4   
84      -122.396401  37.793249 -122.408275  37.778852         28954.0   
159     -122.398298  37.791734 -122.389431  37.788852         28894.3   
...             ...        ...         ...        ...             ...   
2819273 -122.493352  37.853609 -122.519445  37.973755         28842.7   
2819276 -122.480695  37.837944 -122.512136  37.964002         28881.1   
2819312 -122.480319  37.837677 -122.530585  37.898376         28919.9   
2819365 -122.562390  37.899031 -122.525394  37.874967         28956.7   
2819384 -122.570881  37.913599 -122.540335  37.892252         28979.9   

                                   orig_geometry  \
3                POINT (-122.3989918 37.7942922)   
43               PO

In [16]:
morning_peak_od_df = create_od_df(morning_peak)
morning_peak_od_df.head()

,orig,dest,dep_time
3,1.0,2.0,28975.4
43,1.0,5.0,28891.3
57,1.0,5.0,28919.4
84,1.0,11.0,28954.0
159,1.0,16.0,28894.3


In [17]:
new_df = morning_peak_od_df.copy()
new_df.head()

,orig,dest,dep_time
3,1.0,2.0,28975.4
43,1.0,5.0,28891.3
57,1.0,5.0,28919.4
84,1.0,11.0,28954.0
159,1.0,16.0,28894.3


In [18]:
new_df = new_df.dropna()

In [19]:
#create dataframe that contains ODs from randomly assigned node in O taz to randomly assigned node in D taz
start = time.time()
origs, dests, bad_list = assign_node_to_od(new_df, taz_nodes_dict, 'random')
end = time.time()
print("time taken = {}".format(end - start))

number of OD taz with zero nodes = 3
time taken = 0.20865988731384277


In [20]:
print(len(bad_list))

3


In [21]:
final_df = new_df.drop(new_df.index[bad_list])


# Make final dataframe that is in the SAMPN, PERNO, orig, dest format of OD for the traffic microsimulator

In [ ]:
#create a new dataframe with person id, person num, orig OSMID, and dest OSMID
def make_final_df(origs, dests, morning_peak):
    #osmid_o = [x['osmid'] for x in origs]
    #osmid_d = [x['osmid'] for x in dests]
    final_df = pd.DataFrame(columns=['SAMPN', 'PERNO','orig', 'dest'])
    final_df['SAMPN'] = morning_peak['person_id']
    final_df['PERNO'] = 1
    final_df['orig'] = origs
    final_df['dest'] = dests

    return final_df

final_od_file_df = make_final_df(origs, dests, final_df)

In [22]:
#create a new dataframe with person id, person num, orig OSMID, and dest OSMID
def make_final_df_deptime(origs, dests, morning_peak):
    #osmid_o = [x['osmid'] for x in origs]
    #osmid_d = [x['osmid'] for x in dests]
    final_df = pd.DataFrame(columns=['dep_time','orig', 'dest'])
    final_df['dep_time'] = morning_peak['dep_time']
    final_df['origin'] = origs
    final_df['destination'] = dests

    return final_df

final_od_file_df = make_final_df_deptime(origs, dests, final_df)

In [23]:
final_od_file_df.shape

(56441, 5)

# Save the OD file as od_demand.csv to be used in the traffic microsimulator, cb-cities, and static traffic assignment

In [24]:
#save to csv
final_od_file_df.to_csv('../new_full_network/od_demand.csv', index=False, encoding='utf-8')

# UNUSED STUFF #

# Check the osmid in nodes.csv and compare to the osmids of origins and destinations in the final_df dataframe to see which ones are there and which ones are not

In [ ]:
nodes_df = pd.read_csv("../nodes.csv")

In [ ]:
count_vals_orig = final_df.orig.isin(nodes_df.osmid).astype(int)
print("# of origin nodes in OD file not in nodes file = {}".format(count_vals_orig.sum()))

count_vals_dest = final_df.dest.isin(nodes_df.osmid).astype(int)
print("# of dest nodes in OD file not in nodes file = {}".format(count_vals_dest.sum()))
            
#print("orig count not in nodes file = {}".format(orig_count))
#print("dest count not in nodes file = {}".format(dest_count))
#print("total count not in nodes file = {}".format(orig_count + dest_count))

            

In [ ]:
count = 0
for idx, row in final_od_file_df.iterrows():
    if row['orig'] == row['dest']:
        #print(row['orig'], row['dest'])
        count += 1
        
        
print("# of OD pairs with same src and dest = {}".format(count))

In [ ]:
#create dataframe of each person with origin TAZ and destination TAZ
def create_od_df(trips_df):
    orig_dest_df = pd.DataFrame(columns=['orig', 'dest'])
    orig_dest_df['orig'] = morning_peak['orig_taz']
    orig_dest_df['dest'] = morning_peak['dest_taz']
    return orig_dest_df

def assign_node_to_od(od_df, taz_street_network, assign='random'):
    new_df = od_df.copy()
    origs = []
    dests = []
    for index, row in od_df.iterrows():
        #get the lengths of the networks so that we can randomly choose a number in those ranges
        len_network_o = len(taz_graph_list[row['orig']].nodes())
        len_network_d = len(taz_graph_list[row['dest']].nodes())
        
        if assign == 'random':
            #get the randomly chosen value within the length range
            rand_val_o = randint(0, len_network_o - 1)
            rand_val_d = randint(0, len_network_d - 1)
        
        #make the network nodes of origin and destination taz lists
        list_of_taz_nodes_o = list(taz_graph_list[row['orig']].nodes())
        list_of_taz_nodes_d = list(taz_graph_list[row['dest']].nodes())
        #print(list_of_taz_nodes_o)
        
        #get the OSMID of the nodes from the above lists based on the random value as the index
        taz_graph_node_index_o = list_of_taz_nodes_o[rand_val_o]
        taz_graph_node_index_d = list_of_taz_nodes_d[rand_val_d]
        #print(taz_graph_node_index_o)
        
        #set the new nodes as that person's O and D
        node_o = taz_graph_list[row['orig']].node[taz_graph_node_index_o]
        #print(node_o)
        node_d = taz_graph_list[row['dest']].node[taz_graph_node_index_d]
        #print(node_d)
        
        origs += [node_o,]
        dests += [node_d,]
        
    return origs, dests

In [ ]:
#make each TAZ a graph with its street netowrks
taz_graph_list = []
taz_num = 0
start = time.time()
for x in taz['geometry']:
    polygon_hull = x.convex_hull
    polygon_hull_proj, crs = ox.project_geometry(polygon_hull)
    polygon_hull_proj_buff = polygon_hull_proj.buffer(200) #200 meters
    polygon_hull_buff, crs = ox.project_geometry(polygon_hull_proj_buff, crs=crs, to_latlong=True)
    try:
        G = ox.graph_from_polygon(polygon_hull_buff, network_type='drive', simplify=True) 
    except: 
        print('An error occurred.')
    #latlng_geom, _ = ox.project_geometry(taz['geometry'].iloc[0], crs={'init':'epsg:28992'}, to_latlong=True)
    #print("taz num = {}".format(taz_num))
    #print("num nodes = {}".format(len(G.nodes())))
    taz_graph_list += [G,]
    taz_num += 1
end_time = time.time()
print("total time = {}".format(abs(end_time - start_time)))
#ox.plot_graph(G)

In [ ]:
ox.plot_graph(taz_graph_list[400])

In [ ]:
taz_graph_list[400].node[65303936]

# Trying to do the OD assignment in each taz another way

In [ ]:
#create full bay area street network
#identify bay area counties by fips code
bayarea = {'Alameda':'001',
           'Contra Costa':'013',
           'Marin':'041',
           'Napa':'055',
           'San Francisco':'075',
           'San Mateo':'081',
           'Santa Clara':'085',
           'Solano':'095',
           'Sonoma':'097'
          }

In [ ]:
# shapefile of counties
counties_shapefile_dir = 'cb_2016_us_county_500k'
counties = gpd.read_file("{}.shp".format(counties_shapefile_dir))
len(counties)

# retain only those tracts that are in the bay area counties
mask = (counties['STATEFP'] == '06') & (counties['COUNTYFP'].isin(bayarea.values()))
gdf_bay = counties[mask]
len(gdf_bay)

bayarea_polygon = gdf_bay.unary_union



In [ ]:
# get the convex hull, otherwise we'll cut out bridges over the bay
bayarea_polygon_hull = bayarea_polygon.convex_hull
bayarea_polygon_hull_proj, crs = ox.project_geometry(bayarea_polygon_hull)
bayarea_polygon_hull_proj



In [ ]:
# project by a mile to get connectivities surrounding our O-Ds
bayarea_polygon_hull_proj_buff = bayarea_polygon_hull_proj.buffer(1600) #1 mile in meters
bayarea_polygon_hull_buff, crs = ox.project_geometry(bayarea_polygon_hull_proj_buff, crs=crs, to_latlong=True)

In [ ]:
#make overall bay area network
G = ox.graph_from_polygon(bayarea_polygon_hull_buff, network_type='drive', simplify=False)

In [ ]:
# identify all the edge types we want to retain
types = ['motorway', 'motorway_link', 'trunk', 'trunk_link', 
         'primary', 'primary_link', 'secondary', 'secondary_link',
         'tertiary', 'tertiary_link', 'unclassified', 'road']


#types = ['motorway', 'motorway_link', 
#         'primary', 'primary_link', 'secondary', 'secondary_link',
#         'tertiary', 'tertiary_link']

minor_streets = [(u, v, k) for u, v, k, d in G.edges(keys=True, data=True) if d['highway'] not in types]

In [ ]:
# remove minor streets and retain only the largest connected component subgraph
G_ter = G
G_ter.remove_edges_from(minor_streets)
G_ter = ox.remove_isolated_nodes(G_ter)
G_ter_connected = ox.get_largest_component(G_ter, strongly=True)

In [ ]:
# then simplify the graph now that we have only the edge types we want
G_ter_simp = ox.simplify_graph(G_ter_connected, strict=True)
#G_ter_simp = G_ter
#G_ter_simp = G

In [ ]:
taz.iloc[0]

In [ ]:
"""
#plot lat-longs on map
def make_point(row):
    return Point(row.long, row.lat)

# Go through every row, and make a point out of its lat and lon
points = .apply(make_point, axis=1)
"""
def create_lat_long_from_graph_nodes(G):
    G_nodes_list = list(G.nodes())
    lat_list = []
    long_list = []
    osmid_list = []
    for x in G_nodes_list:
        lat_list += [G.nodes()[x]['x'],]
        long_list += [G.nodes()[x]['y'],]
        osmid_list += [x,]
        
    return lat_list, long_list, osmid_list

lat_list, long_list, osmid_list = create_lat_long_from_graph_nodes(G_ter_simp)


def make_df_lat_long_osmid(lat, long, osmid):
    final_df = pd.DataFrame(columns=['osmid', 'lat','long'])
    final_df['osmid'] = osmid
    final_df['lat'] = lat
    final_df['long'] = long
    
    return final_df

nodes_df = make_df_lat_long_osmid(lat_list, long_list, osmid_list)
    


#G_ter_simp.nodes()[1377399032]


In [ ]:
nodes_df.head()

In [ ]:
#plot lat-longs on map
def make_point(row):
    return Point(row.long, row.lat)

# Go through every row, and make a point out of its lat and lon
points = nodes_df.apply(make_point, axis=1)

nodes_points = gpd.GeoDataFrame(nodes_df, geometry=points)
nodes_points.head()

In [ ]:
taz.iloc[0]

In [ ]:
merged_taz_1 = gpd.sjoin(nodes_points, taz.iloc[10], how="left", op="contains")
merged_taz_1.head()
#taz.iloc[0].contains(nodes_points)


In [ ]:
#get centroid coordinates of each TAZ (x and y)
lats = taz.geometry.centroid.x
longs = taz.geometry.centroid.y

In [ ]:
taz.iloc[10]['geometry']

In [ ]:
def make_taz_lat_long_df(tazs, lat, long):
    new_df = pd.DataFrame(columns=['taz', 'lat','long'])
    new_df['taz'] = tazs
    new_df['lat'] = lats
    new_df['long'] = longs
    return new_df

centroid_lat_long_df = make_taz_lat_long_df(tazs, lats, longs)

In [ ]:
#plot lat-longs on map
def make_point(row):
    return Point(row.long, row.lat)

# Go through every row, and make a point out of its lat and lon
points = centroid_lat_long_df.apply(make_point, axis=1)

# Make a new GeoDataFrame
# using the data from our old df
# but also adding in the geometry we just made
taz_centroids = gpd.GeoDataFrame(centroid_lat_long_df, geometry=points)

# It doesn't come with a CRS because it's a CSV, so let's
# say "hey, let's use the standard shape of the earth etc"
taz_centroids.crs = {'init': 'epsg:4326'}
    

#look at the first few
taz_centroids.head(n=100)

In [ ]:
#plot taz centroids lat-long
ax = taz_centroids.plot(figsize=(25,25), markersize=20, color='green', alpha=0.75)
ax.axis('on')

In [ ]:
#taz_loc_df.sort_values('taz')

In [ ]:
taz['geometry']

In [ ]:
"""
def parse_taz_loc_df_into_lat_long(taz):
    
    new_df = pd.DataFrame(columns=['taz','lat','long'])
    
    for x in taz:
        new_df = pd.DataFrame(columns=['taz','lat','long'])
        new_df['taz'] = x['taz']
        new_df['lat'] = = taz['geometry'][0].split(',')
    
        #remove 'POLYGON' from first element
        new_str[0] = new_str[0].split('N (')[1]

        #get longitude for all values in polygon
        new_str_list = [x.split(' ') for x in new_str]
        #print(new_str_list)

        #remove first element
        del new_str_list[0]

        #remove last element
        del new_str_list[-1]

        #get lat and long
        long = [float(x[1]) for x in new_str_list]
        lat = [float(x[2]) for x in new_str_list]
        #print(new_str[3].split(' '))
        #print(long)
        #print(lat)
        return long, lat


def make_lat_long_df(lat, long):
    new_df = pd.DataFrame(columns=['lat','long'])
    #new_df['segment_id'] = df['PublicSegID']
    new_df['lat'] = lat
    new_df['long'] = long
    return new_df


lat, long = parse_taz_loc_df_into_lat_long(taz_loc_df)
lat_long_df = make_lat_long_df(lat, long)
lat_long_df
"""